In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import json

In [ ]:

# Define the relative path to the image
image_path = './gtFine_trainvaltest/gtFine/train/aachen/aachen_000000_000019_gtFine_color.png'

# Check if the file exists in the specified path
image = Image.open(image_path)

# Convert the image to a numpy array
image_array = np.array(image)

# Visualize the image using matplotlib
plt.imshow(image_array)
plt.axis('off')  # Hide axes
plt.show()

# Display basic information about the image array
print("Image array shape:", image_array.shape)
print("Image array dtype:", image_array.dtype)



In [ ]:
# Define the base path to your dataset (adjust as necessary)
base_path = './gtFine_trainvaltest/gtFine/train/'

# List all cities (directories) in the base train directory
cities = os.listdir(base_path)

# Loop through each city
for city in cities:
    city_path = os.path.join(base_path, city)
    
    if os.path.isdir(city_path):
        # List all images in the city's directory (assuming they follow the naming pattern)
        image_files = [f for f in os.listdir(city_path) if f.endswith('_gtFine_color.png')]

        for image_file in image_files:
            image_path = os.path.join(city_path, image_file)
            instance_path = image_path.replace('_gtFine_color.png', '_gtFine_instanceIds.png')
            label_path = image_path.replace('_gtFine_color.png', '_gtFine_labelIds.png')
            json_path = image_path.replace('_gtFine_color.png', '_gtFine_polygons.json')

            # Open the color image, instance image, and label image
            if os.path.exists(image_path):
                color_image = Image.open(image_path)
                color_array = np.array(color_image)

                # Optionally load the instance and label images
                instance_image = Image.open(instance_path) if os.path.exists(instance_path) else None
                label_image = Image.open(label_path) if os.path.exists(label_path) else None
                
                # Optionally load the JSON if you need the polygon data
                if os.path.exists(json_path):
                    with open(json_path) as json_file:
                        polygons = json.load(json_file)
                
                # Visualize the color image (without normalization)
                plt.imshow(color_array)  # No need for normalization here
                plt.axis('off')  # Hide axes
                plt.show()

                # Additional processing with instance, label images or polygons can be done here.
                
                print(f"Loaded: {image_file}")
            else:
                print(f"Image not found: {image_path}")


In [ ]:
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import layers
import tensorflow as tf

In [ ]:
# Path to the dataset
base_path = './gtFine/gtFine/train/'

# Define the class ID for 'road' (for example, 'road' has label ID 7 in Cityscapes dataset)
road_class_id = 7

# Function to load and preprocess image and its binary mask (road vs non-road)
def load_data(image_path, label_path, target_size=(256, 256)):
    # Check if paths exist (to avoid errors)
    if not os.path.exists(image_path) or not os.path.exists(label_path):
        raise ValueError(f"Image or label path does not exist: {image_path}, {label_path}")
    
    # Load and resize the image
    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image) / 255.0  # Normalize the image to [0, 1]
    
    # Load the label image (class ID map) and convert it to a binary mask (1 for road, 0 for non-road)
    label_image = load_img(label_path, target_size=target_size, color_mode='grayscale')
    label_array = img_to_array(label_image)
    
    # Create a binary mask: 1 for road (ID 7), 0 for non-road
    binary_mask = (label_array == road_class_id).astype(np.uint8)
    
    return image, binary_mask


In [ ]:

base_path = './gtFine/gtFine/train/'
image_path = os.path.join(base_path, 'aachen/aachen_000000_000019_gtFine_color.png')
label_path = os.path.join(base_path, 'aachen/aachen_000000_000019_gtFine_labelIds.png')
load_data(image_path, label_path)


In [ ]:
# # Function to build U-Net model for binary segmentation
# def build_unet(input_shape):
#     inputs = tf.keras.Input(input_shape)
    
#     # Encoder (Downsampling)
#     c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
#     c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
#     p1 = layers.MaxPooling2D((2, 2))(c1)
    
#     # Decoder (Upsampling)
#     u1 = layers.UpSampling2D((2, 2))(p1)
#     u1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u1)
#     u1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u1)
    
#     # Output layer (Binary mask)
#     outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(u1)  # Binary output (0 or 1)
    
#     model = tf.keras.Model(inputs, outputs)
#     return model

# # Set up your U-Net model
# input_shape = (256, 256, 3)  # Adjust this based on your image size
# model = build_unet(input_shape)
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # Load your training data
# def load_all_images_and_labels(base_path, target_size=(256, 256)):
#     # Loop through all cities
#     cities = os.listdir(base_path)
#     X_train, y_train = [], []

#     for city in cities:
#         city_path = os.path.join(base_path, city)
        
#         if os.path.isdir(city_path):
#             # List all images in the city's directory
#             image_files = [f for f in os.listdir(city_path) if f.endswith('_gtFine_color.png')]

#             for image_file in image_files:
#                 image_path = os.path.join(city_path, image_file)
#                 label_path = image_path.replace('_gtFine_color.png', '_gtFine_labelIds.png')

#                 if os.path.exists(image_path) and os.path.exists(label_path):
#                     # Load and preprocess the data
#                     image, binary_mask = load_data(image_path, label_path, target_size)
#                     X_train.append(image)
#                     y_train.append(binary_mask)

#     return np.array(X_train), np.array(y_train)

# # Load all images and labels for training
# X_train, y_train = load_all_images_and_labels(base_path)

# # Train the model (adjust epochs and batch size as needed)
# model.fit(X_train, y_train, epochs=10, batch_size=32)

# # Function to make predictions on all images
# def predict_on_all_images(base_path, model, target_size=(256, 256)):
#     # Loop through all cities again
#     cities = os.listdir(base_path)
    
#     for city in cities:
#         city_path = os.path.join(base_path, city)
        
#         if os.path.isdir(city_path):
#             # List all images in the city's directory
#             image_files = [f for f in os.listdir(city_path) if f.endswith('_gtFine_color.png')]

#             for image_file in image_files:
#                 image_path = os.path.join(city_path, image_file)

#                 if os.path.exists(image_path):
#                     # Load and preprocess the image (no need to load the labels for prediction)
#                     test_image = load_img(image_path, target_size=target_size)
#                     test_image = img_to_array(test_image) / 255.0  # Normalize the image
#                     test_image = np.expand_dims(test_image, axis=0)  # Add batch dimension
                    
#                     # Make the prediction
#                     predicted_mask = model.predict(test_image)[0]  # Get the prediction for the first image
#                     binary_predicted_mask = (predicted_mask > 0.5).astype(np.uint8)
                    
#                     # Visualize or save the predicted binary mask (road vs non-road)
#                     plt.imshow(binary_predicted_mask, cmap='gray')
#                     plt.title(f"Predicted Mask for {image_file}")
#                     plt.axis('off')
#                     plt.show()

# # Use the trained model to make predictions on all images
# predict_on_all_images(base_path, model)

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.ensemble import RandomForestRegressor  # or use XGBRegressor from xgboost
from sklearn.metrics import mean_squared_error

In [ ]:
def load_all_images_and_labels(base_path, target_size=(256, 256)):
    cities = os.listdir(base_path)
    X_train, y_train = [], []

    for city in cities:
        city_path = os.path.join(base_path, city)
        
        if os.path.isdir(city_path):
            # List all images in the city's directory
            image_files = [f for f in os.listdir(city_path) if f.endswith('_gtFine_color.png')]

            for image_file in image_files:
                image_path = os.path.join(city_path, image_file)
                label_path = image_path.replace('_gtFine_color.png', '_gtFine_labelIds.png')

                if os.path.exists(image_path) and os.path.exists(label_path):
                    # Load and preprocess the data
                    image, binary_mask = load_data(image_path, label_path, target_size)
                    
                    # Flatten the image and binary mask for regression
                    X_train.append(image.flatten())  # Flatten the image into a 1D vector
                    y_train.append(binary_mask.flatten())  # Flatten the mask into a 1D vector

    return np.array(X_train), np.array(y_train)

# Load all images and labels for training
X_train, y_train = load_all_images_and_labels(base_path)

# Initialize and train a regression model (e.g., Random Forest Regressor)
regressor = RandomForestRegressor(n_estimators=100, random_state=42)
regressor.fit(X_train, y_train)

# Function to make predictions on all images using the regressor
def predict_on_all_images(base_path, model, target_size=(256, 256)):
    cities = os.listdir(base_path)
    
    for city in cities:
        city_path = os.path.join(base_path, city)
        
        if os.path.isdir(city_path):
            image_files = [f for f in os.listdir(city_path) if f.endswith('_gtFine_color.png')]

            for image_file in image_files:
                image_path = os.path.join(city_path, image_file)

                if os.path.exists(image_path):
                    # Load and preprocess the image (no need to load the labels for prediction)
                    test_image = load_img(image_path, target_size=target_size)
                    test_image = img_to_array(test_image) / 255.0  # Normalize the image
                    test_image = test_image.flatten().reshape(1, -1)  # Flatten the image and reshape

                    # Make the prediction
                    predicted_mask = model.predict(test_image)[0]  # Get the prediction for the first image
                    
                    # Reshape the prediction back to the original image size (for visualization)
                    predicted_mask = predicted_mask.reshape(target_size)  # Reshape back to (256, 256)
                    
                    # Visualize the predicted binary mask
                    binary_predicted_mask = (predicted_mask > 0.5).astype(np.uint8)  # Apply a threshold to get a binary mask
                    plt.imshow(binary_predicted_mask, cmap='gray')
                    plt.title(f"Predicted Mask for {image_file}")
                    plt.axis('off')
                    plt.show()

# Use the trained model to make predictions on all images
predict_on_all_images(base_path, regressor)